# Machine-learning Notebook
- preceeded by feature creation notebooks

In [1]:
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt
%matplotlib inline
import welly
from welly import Well
import lasio
import glob
from sklearn import neighbors
import pickle
import math
import dask
import dask.dataframe as dd
from dask.distributed import Client
# import pdvega
# import vega
import random
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import mean_squared_error


In [2]:
print(welly.__version__)
print(dask.__version__)
print(pd.__version__)

0.3.5
0.18.1
0.23.1


In [3]:
%%timeit
import os
env = %env

79.1 µs ± 1.37 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [4]:
#### Had to change display options to get this to print in full!
#pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.max_colwidth = 100000

In [5]:
knn_dir = "../WellsKNN/"
load_dir = "../loadLAS"
features_dir = "../createFeatures/"

## If you open this notebook fresh and jump to a point below where a pick file is read in, you still need to load everything above! 

-------------

------------

## Reading in the last hdf5 file

In [6]:
h5_to_load = 'df_all_wells_wKNN_DEPTHtoDEPT_KNN1PredTopMcM_20180724.h5'
h5_key = 'df'
df_all_Col_preSplit = pd.read_hdf(features_dir+h5_to_load, h5_key)

In [7]:
df_all_Col_preSplit.head()

,CALI,COND,DELT,DENS,DEPT,DEPTH,DPHI,DPHI:1,DPHI:2,DT,GR,GR:1,GR:2,IL,ILD,ILD:1,ILD:2,ILM,LITH,LLD,LLS,NPHI,PHID,PHIN,RESD,RHOB,RT,SFL,SFLU,SN,SNP,SP,UWI,SitID,McMurray_Base_HorID,McMurray_Top_HorID,McMurray_Base_DEPTH,McMurray_Top_DEPTH,McMurray_Base_Qual,McMurray_Top_Qual,lat,lng,NN1_McMurray_Top_DEPTH,NN1_McMurray_Base_DEPTH,NN1_thickness,MM_Top_Depth_predBy_NN1thick,HorID,Pick,Quality,HorID_paleoz,Pick_paleoz,Quality_paleoz,diff_TMcM_Pick_v_DEPT,diff_TPal_Pick_v_DEPT,cat_isTopMcMrNearby_known,cat_isTopPalNearby_known,DistFrom_NN1_TopDepth_Abs,NewWell,LastBitWell,TopWellDept,BotWellDept,FromTopWell,FromBotWell,WellThickness,closerToBotOrTop,closTopBotDist,rowsToEdge,GR_min_5winSize_dirAroundMin,GR_min_5winSize_dirAboveMin,GR_min_5winSize_dirAroundMax,GR_min_5winSize_dirAboveMax,GR_min_5winSize_dirAroundMean,GR_min_5winSize_dirAboveMean,GR_min_5winSize_dirAbovenLarge,GR_min_5winSize_dirAroundnLarge,GR_min_7winSize_dirAroundMin,GR_min_7winSize_dirAboveMin,GR_min_7winSize_dirAroundMax,GR_min_7winSize_dirAboveMax,GR_min_7winSize_dirAroundMean,GR_min_7winSize_dirAboveMean,GR_min_7winSize_dirAbovenLarge,GR_min_7winSize_dirAroundnLarge,GR_min_11winSize_dirAroundMin,GR_min_11winSize_dirAboveMin,GR_min_11winSize_dirAroundMax,GR_min_11winSize_dirAboveMax,GR_min_11winSize_dirAroundMean,GR_min_11winSize_dirAboveMean,GR_min_11winSize_dirAbovenLarge,GR_min_11winSize_dirAroundnLarge,GR_min_21winSize_dirAroundMin,GR_min_21winSize_dirAboveMin,GR_min_21winSize_dirAroundMax,GR_min_21winSize_dirAboveMax,GR_min_21winSize_dirAroundMean,GR_min_21winSize_dirAboveMean,GR_min_21winSize_dirAbovenLarge,GR_min_21winSize_dirAroundnLarge,ILD_min_5winSize_dirAroundMin,ILD_min_5winSize_dirAboveMin,ILD_min_5winSize_dirAroundMax,ILD_min_5winSize_dirAboveMax,ILD_min_5winSize_dirAroundMean,ILD_min_5winSize_dirAboveMean,ILD_min_5winSize_dirAbovenLarge,ILD_min_5winSize_dirAroundnLarge,ILD_min_7winSize_dirAroundMin,ILD_min_7winSize_dirAboveMin,ILD_min_7winSize_dirAroundMax,ILD_min_7winSize_dirAboveMax,ILD_min_7winSize_dirAroundMean,ILD_min_7winSize_dirAboveMean,ILD_min_7winSize_dirAbovenLarge,ILD_min_7winSize_dirAroundnLarge,ILD_min_11winSize_dirAroundMin,ILD_min_11winSize_dirAboveMin,ILD_min_11winSize_dirAroundMax,ILD_min_11winSize_dirAboveMax,ILD_min_11winSize_dirAroundMean,ILD_min_11winSize_dirAboveMean,ILD_min_11winSize_dirAbovenLarge,ILD_min_11winSize_dirAroundnLarge,ILD_min_21winSize_dirAroundMin,ILD_min_21winSize_dirAboveMin,ILD_min_21winSize_dirAroundMax,ILD_min_21winSize_dirAboveMax,ILD_min_21winSize_dirAroundMean,ILD_min_21winSize_dirAboveMean,ILD_min_21winSize_dirAbovenLarge,ILD_min_21winSize_dirAroundnLarge
0,167.003,NaN,NaN,NaN,149.602,NaN,0.227,NaN,NaN,NaN,102.473,NaN,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN,NaN,0.460,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00/10-32-080-20W4/0,112385,14000,13000,384.66,377.95,1,3,55.978836,-113.095365,389.0,414.0,25.0,359.66,13000,377.95,3,14000,384.66,1,228.348,235.058,0,0,210.058,True,False,149.602,396.102,0.00,246.50,246.5,FromTopWell,0.00,0,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,199.159,NaN,NaN,NaN,149.852,NaN,0.263,NaN,NaN,NaN,122.589,NaN,NaN,NaN,4.202,NaN,NaN,NaN,NaN,NaN,NaN,0.550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00/10-32-080-20W4/0,112385,14000,13000,384.66,377.95,1,3,55.978836,-113.095365,389.0,414.0,25.0,359.66,13000,377.95,3,14000,384.66,1,228.098,234.808,0,0,209.808,False,False,149.602,396.102,0.25,246.25,246.5,FromTopWell,0.25,1,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589

In [8]:
# df_all_wells_wKNN_DEPTHtoDEPT_KNN1PredTopMcM_20180724_initial.head()

-------------

## Because we have a lot more data points away from the pick (one class) than exactly at the pick or close to the pick (another class), we run the risk of being class heavy. We'll attemp to deal with this problem by only using some of the data not near or at a pick

Get all the UWIs

In [9]:
UWIs = list(df_all_Col_preSplit['UWI'].unique())

Find the number of wells if you want 80%

In [10]:
numberOfTrainingWells = math.floor(len(UWIs)*0.8)
numberOfTrainingWells

1525

Randomly select that number of UWIs for training and the ones left for test

In [11]:
UWIs_training = random.sample(UWIs, numberOfTrainingWells)

In [12]:
UWIs_test = [x for x in UWIs if x not in UWIs_training]

In [13]:
print("train",len(UWIs_training))
print("test",len(UWIs_test))

train 1525
test 382


In [14]:
df_all_Col_preSplit_wTrainTest = df_all_Col_preSplit.copy()

In [15]:
df_all_Col_preSplit_wTrainTest['trainOrTest'] = np.where(df_all_Col_preSplit_wTrainTest['UWI'].isin(UWIs_training), 'train', 'test')

In [16]:
df_all_Col_preSplit_wTrainTest.tail()

,CALI,COND,DELT,DENS,DEPT,DEPTH,DPHI,DPHI:1,DPHI:2,DT,GR,GR:1,GR:2,IL,ILD,ILD:1,ILD:2,ILM,LITH,LLD,LLS,NPHI,PHID,PHIN,RESD,RHOB,RT,SFL,SFLU,SN,SNP,SP,UWI,SitID,McMurray_Base_HorID,McMurray_Top_HorID,McMurray_Base_DEPTH,McMurray_Top_DEPTH,McMurray_Base_Qual,McMurray_Top_Qual,lat,lng,NN1_McMurray_Top_DEPTH,NN1_McMurray_Base_DEPTH,NN1_thickness,MM_Top_Depth_predBy_NN1thick,HorID,Pick,Quality,HorID_paleoz,Pick_paleoz,Quality_paleoz,diff_TMcM_Pick_v_DEPT,diff_TPal_Pick_v_DEPT,cat_isTopMcMrNearby_known,cat_isTopPalNearby_known,DistFrom_NN1_TopDepth_Abs,NewWell,LastBitWell,TopWellDept,BotWellDept,FromTopWell,FromBotWell,WellThickness,closerToBotOrTop,closTopBotDist,rowsToEdge,GR_min_5winSize_dirAroundMin,GR_min_5winSize_dirAboveMin,GR_min_5winSize_dirAroundMax,GR_min_5winSize_dirAboveMax,GR_min_5winSize_dirAroundMean,GR_min_5winSize_dirAboveMean,GR_min_5winSize_dirAbovenLarge,GR_min_5winSize_dirAroundnLarge,GR_min_7winSize_dirAroundMin,GR_min_7winSize_dirAboveMin,GR_min_7winSize_dirAroundMax,GR_min_7winSize_dirAboveMax,GR_min_7winSize_dirAroundMean,GR_min_7winSize_dirAboveMean,GR_min_7winSize_dirAbovenLarge,GR_min_7winSize_dirAroundnLarge,GR_min_11winSize_dirAroundMin,GR_min_11winSize_dirAboveMin,GR_min_11winSize_dirAroundMax,GR_min_11winSize_dirAboveMax,GR_min_11winSize_dirAroundMean,GR_min_11winSize_dirAboveMean,GR_min_11winSize_dirAbovenLarge,GR_min_11winSize_dirAroundnLarge,GR_min_21winSize_dirAroundMin,GR_min_21winSize_dirAboveMin,GR_min_21winSize_dirAroundMax,GR_min_21winSize_dirAboveMax,GR_min_21winSize_dirAroundMean,GR_min_21winSize_dirAboveMean,GR_min_21winSize_dirAbovenLarge,GR_min_21winSize_dirAroundnLarge,ILD_min_5winSize_dirAroundMin,ILD_min_5winSize_dirAboveMin,ILD_min_5winSize_dirAroundMax,ILD_min_5winSize_dirAboveMax,ILD_min_5winSize_dirAroundMean,ILD_min_5winSize_dirAboveMean,ILD_min_5winSize_dirAbovenLarge,ILD_min_5winSize_dirAroundnLarge,ILD_min_7winSize_dirAroundMin,ILD_min_7winSize_dirAboveMin,ILD_min_7winSize_dirAroundMax,ILD_min_7winSize_dirAboveMax,ILD_min_7winSize_dirAroundMean,ILD_min_7winSize_dirAboveMean,ILD_min_7winSize_dirAbovenLarge,ILD_min_7winSize_dirAroundnLarge,ILD_min_11winSize_dirAroundMin,ILD_min_11winSize_dirAboveMin,ILD_min_11winSize_dirAroundMax,ILD_min_11winSize_dirAboveMax,ILD_min_11winSize_dirAroundMean,ILD_min_11winSize_dirAboveMean,ILD_min_11winSize_dirAbovenLarge,ILD_min_11winSize_dirAroundnLarge,ILD_min_21winSize_dirAroundMin,ILD_min_21winSize_dirAboveMin,ILD_min_21winSize_dirAroundMax,ILD_min_21winSize_dirAboveMax,ILD_min_21winSize_dirAroundMean,ILD_min_21winSize_dirAboveMean,ILD_min_21winSize_dirAbovenLarge,ILD_min_21winSize_dirAroundnLarge,trainOrTest
1482751,NaN,NaN,NaN,NaN,359.00,NaN,0.014,NaN,NaN,NaN,61.724,NaN,NaN,NaN,53.940,NaN,NaN,NaN,NaN,NaN,NaN,0.191,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00/10-35-081-15W4/0,154240,14000,13000,348.0,321.0,1,3,56.066128,-112.234008,300.5,323.5,23.0,325.0,13000,321.0,3,14000,348.0,1,-38.00,-11.00,0,0,34.00,False,False,140.0,360.0,219.00,1.00,220.0,FromBotWell,1.00,4,61.724,61.724,61.724,61.724,61.724,61.724,61.724,61.724,61.724,61.724,61.724,61.724,61.724,61.724,61.724,61.724,61.724,61.724,61.724,61.724,61.724,61.724,61.724,61.724,61.724,61.724,61.724,61.724,61.724,61.724,61.724,61.724,53.940,53.940,53.940,53.940,53.940,53.940,53.940,53.940,53.940,53.940,53.940,53.940,53.940,53.940,53.940,53.940,53.940,53.940,53.940,53.940,53.940,53.940,53.940,53.940,53.940,53.940,53.940,53.940,53.940,53.940,53.940,53.940,train
1482752,NaN,NaN,NaN,NaN,359.25,NaN,0.014,NaN,NaN,NaN,59.927,NaN,NaN,NaN,63.882,NaN,NaN,NaN,NaN,NaN,NaN,0.167,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00/10-35-081-15W4/0,154240,14000,13000,348.0,321.0,1,3,56.066128,-112.234008,300.5,323.5,23.0,325.0,13000,321.0,3,14000,348.0,1,-38.25,-11.25,0,0,34.25,False,False,140.0,360.0,219.25,0.75,220.0,FromBotWell,0.75,3,59.927,59.927,59.927,59.927,59.927,59.927,59.927,59.927,59.927,59.927,59.927,59.927,59.927,59.927,59.927,59.927,59.927,59.927,59.927,59.927,59.927,59.927,59.927,59.927,59.927,59.92

In [17]:
df_test_5 = df_all_Col_preSplit_wTrainTest.copy()

### Additionally, we want to split such:
- balance class sizes somewhat otherwise rows not near the pick will dominate the population

- create new dataframe that keeps every row that is classed at or near the pick but for rows outside that area only keeps a some fraction based on remainder after division?

In [18]:
def countRowsByClassOfNearPickOrNot(df,arrayOfClass,divisionInt,classToShrink):
    for eachClass in arrayOfClass:
        print("length of rows with "+str(eachClass)+" in cat_isTopMcMrNearby_known:",len(df[df['cat_isTopMcMrNearby_known'] == eachClass]))
    df_NearPickZeroSmall = df.loc[(df.index%10 != 3) & (df['cat_isTopMcMrNearby_known'] == classToShrink)]
    print("length of rows with 0 in cat_isTopMcMrNearby_known and %"+str(divisionInt)+" == 0 is:",len(df_NearPickZeroSmall))
    print("% reduction in classs 0 is:", math.floor(len(df_NearPickZeroSmall) / len(df['cat_isTopMcMrNearby_known'] == classToShrink) * 100),"%")
    total_after_reduction_in_bigger_class = len(df[df['cat_isTopMcMrNearby_known'] == classToShrink]) -len(df_NearPickZeroSmall)
    print("if taken out using this remainder, the total number of 0 class will be: ",total_after_reduction_in_bigger_class)
#     print("ratio between that class away from pick and classes near pick is :":)
    return df_NearPickZeroSmall

In [19]:
class_array_NearPick = [100,95,60,0]
test_df_return = countRowsByClassOfNearPickOrNot(df_test_5,class_array_NearPick,2,0)
# test_df_return[0:200]['DEPT']

length of rows with 100 in cat_isTopMcMrNearby_known: 1014
length of rows with 95 in cat_isTopMcMrNearby_known: 4819
length of rows with 60 in cat_isTopMcMrNearby_known: 61345
length of rows with 0 in cat_isTopMcMrNearby_known: 1415578
length of rows with 0 in cat_isTopMcMrNearby_known and %2 == 0 is: 1274047
% reduction in classs 0 is: 85 %
if taken out using this remainder, the total number of 0 class will be:  141531


In [20]:
def dropsRowsWithMatchClassAndDeptRemainderIsZero(df,Col,RemainderInt,classToShrink):
    """
    Takes as input a dataframe, a column, a remainder integer, and a class within the column.
    Returns the dataframe minus the rows that match the ClassToShrink in the Col and have a depth from the DEPT col with a remainder of zero.
    """
    print("original lenght of dataframe = ",len(df))
    df_new = df.drop(df[(df[Col] == classToShrink) & (df.index%10 != 0)].index)
    print("length of new dataframe after dropping rows = ",len(df_new))
    print("number of rows dropped = ",len(df)-len(df_new))
    return df_new

In [21]:
df_all_Col_preSplit_wTrainTest_ClassBalanced = dropsRowsWithMatchClassAndDeptRemainderIsZero(df_all_Col_preSplit_wTrainTest,'cat_isTopMcMrNearby_known',7,0)

original lenght of dataframe =  1482756
length of new dataframe after dropping rows =  208758
number of rows dropped =  1273998


In [22]:
df_all_Col_preSplit_wTrainTest_ClassBalanced.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 208758 entries, 0 to 1482750
Columns: 132 entries, CALI to trainOrTest
dtypes: bool(2), float64(115), int64(12), object(3)
memory usage: 209.0+ MB


## Identify which columns to use as features 

In [23]:
col_list = df_all_Col_preSplit_wTrainTest_ClassBalanced.columns
print(col_list)

Index(['CALI', 'COND', 'DELT', 'DENS', 'DEPT', 'DEPTH', 'DPHI', 'DPHI:1', 'DPHI:2', 'DT',
       ...
       'ILD_min_11winSize_dirAroundnLarge', 'ILD_min_21winSize_dirAroundMin', 'ILD_min_21winSize_dirAboveMin', 'ILD_min_21winSize_dirAroundMax', 'ILD_min_21winSize_dirAboveMax', 'ILD_min_21winSize_dirAroundMean', 'ILD_min_21winSize_dirAboveMean', 'ILD_min_21winSize_dirAbovenLarge', 'ILD_min_21winSize_dirAroundnLarge', 'trainOrTest'], dtype='object', length=132)


In [24]:
df_all_Col_preSplit_wTrainTest_ClassBalanced.head()

,CALI,COND,DELT,DENS,DEPT,DEPTH,DPHI,DPHI:1,DPHI:2,DT,GR,GR:1,GR:2,IL,ILD,ILD:1,ILD:2,ILM,LITH,LLD,LLS,NPHI,PHID,PHIN,RESD,RHOB,RT,SFL,SFLU,SN,SNP,SP,UWI,SitID,McMurray_Base_HorID,McMurray_Top_HorID,McMurray_Base_DEPTH,McMurray_Top_DEPTH,McMurray_Base_Qual,McMurray_Top_Qual,lat,lng,NN1_McMurray_Top_DEPTH,NN1_McMurray_Base_DEPTH,NN1_thickness,MM_Top_Depth_predBy_NN1thick,HorID,Pick,Quality,HorID_paleoz,Pick_paleoz,Quality_paleoz,diff_TMcM_Pick_v_DEPT,diff_TPal_Pick_v_DEPT,cat_isTopMcMrNearby_known,cat_isTopPalNearby_known,DistFrom_NN1_TopDepth_Abs,NewWell,LastBitWell,TopWellDept,BotWellDept,FromTopWell,FromBotWell,WellThickness,closerToBotOrTop,closTopBotDist,rowsToEdge,GR_min_5winSize_dirAroundMin,GR_min_5winSize_dirAboveMin,GR_min_5winSize_dirAroundMax,GR_min_5winSize_dirAboveMax,GR_min_5winSize_dirAroundMean,GR_min_5winSize_dirAboveMean,GR_min_5winSize_dirAbovenLarge,GR_min_5winSize_dirAroundnLarge,GR_min_7winSize_dirAroundMin,GR_min_7winSize_dirAboveMin,GR_min_7winSize_dirAroundMax,GR_min_7winSize_dirAboveMax,GR_min_7winSize_dirAroundMean,GR_min_7winSize_dirAboveMean,GR_min_7winSize_dirAbovenLarge,GR_min_7winSize_dirAroundnLarge,GR_min_11winSize_dirAroundMin,GR_min_11winSize_dirAboveMin,GR_min_11winSize_dirAroundMax,GR_min_11winSize_dirAboveMax,GR_min_11winSize_dirAroundMean,GR_min_11winSize_dirAboveMean,GR_min_11winSize_dirAbovenLarge,GR_min_11winSize_dirAroundnLarge,GR_min_21winSize_dirAroundMin,GR_min_21winSize_dirAboveMin,GR_min_21winSize_dirAroundMax,GR_min_21winSize_dirAboveMax,GR_min_21winSize_dirAroundMean,GR_min_21winSize_dirAboveMean,GR_min_21winSize_dirAbovenLarge,GR_min_21winSize_dirAroundnLarge,ILD_min_5winSize_dirAroundMin,ILD_min_5winSize_dirAboveMin,ILD_min_5winSize_dirAroundMax,ILD_min_5winSize_dirAboveMax,ILD_min_5winSize_dirAroundMean,ILD_min_5winSize_dirAboveMean,ILD_min_5winSize_dirAbovenLarge,ILD_min_5winSize_dirAroundnLarge,ILD_min_7winSize_dirAroundMin,ILD_min_7winSize_dirAboveMin,ILD_min_7winSize_dirAroundMax,ILD_min_7winSize_dirAboveMax,ILD_min_7winSize_dirAroundMean,ILD_min_7winSize_dirAboveMean,ILD_min_7winSize_dirAbovenLarge,ILD_min_7winSize_dirAroundnLarge,ILD_min_11winSize_dirAroundMin,ILD_min_11winSize_dirAboveMin,ILD_min_11winSize_dirAroundMax,ILD_min_11winSize_dirAboveMax,ILD_min_11winSize_dirAroundMean,ILD_min_11winSize_dirAboveMean,ILD_min_11winSize_dirAbovenLarge,ILD_min_11winSize_dirAroundnLarge,ILD_min_21winSize_dirAroundMin,ILD_min_21winSize_dirAboveMin,ILD_min_21winSize_dirAroundMax,ILD_min_21winSize_dirAboveMax,ILD_min_21winSize_dirAroundMean,ILD_min_21winSize_dirAboveMean,ILD_min_21winSize_dirAbovenLarge,ILD_min_21winSize_dirAroundnLarge,trainOrTest
0,167.003,NaN,NaN,NaN,149.602,NaN,0.227,NaN,NaN,NaN,102.473,NaN,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN,NaN,0.460,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00/10-32-080-20W4/0,112385,14000,13000,384.66,377.95,1,3,55.978836,-113.095365,389.0,414.0,25.0,359.66,13000,377.95,3,14000,384.66,1,228.348,235.058,0,0,210.058,True,False,149.602,396.102,0.0,246.5,246.5,FromTopWell,0.0,0,102.473,102.473,102.473,102.473,102.4730,102.4730,102.4730,102.4730,102.473,102.473,102.473,102.473,102.473000,102.473000,102.4730,102.4730,102.473,102.473,102.473,102.473,102.473000,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,0.000,0.000,0.000,0.000,0.0000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000,0.000000,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000,0.000000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,train
10,166.675,NaN,NaN,NaN,152.102,NaN,0.269,NaN,NaN,NaN,26.625,NaN,NaN,NaN,30.179,NaN,NaN,NaN,NaN,NaN,NaN,0.355,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00/10-32-080-20W4/0,112385,14000,13000,384.66,377.95,1,3,55.978836,-113.095365,389.0,414.0,25.0,359.66,13000,377.95,3,14000,384.66,1,225.848,232.558,0,0,207.558,False,False,149.602,396.102,2.5,244.0,246.5,FromTopWell,2.5,10,25.825,26.625,50.213,26.625,32.7680,26.6250,26.6250,26.6250,26.625,26.625,26.625,26.625,26.625000,26.625000,26.6250,26.6250,26.625,

In [25]:
col_list = list(col_list)
col_list

['CALI',
 'COND',
 'DELT',
 'DENS',
 'DEPT',
 'DEPTH',
 'DPHI',
 'DPHI:1',
 'DPHI:2',
 'DT',
 'GR',
 'GR:1',
 'GR:2',
 'IL',
 'ILD',
 'ILD:1',
 'ILD:2',
 'ILM',
 'LITH',
 'LLD',
 'LLS',
 'NPHI',
 'PHID',
 'PHIN',
 'RESD',
 'RHOB',
 'RT',
 'SFL',
 'SFLU',
 'SN',
 'SNP',
 'SP',
 'UWI',
 'SitID',
 'McMurray_Base_HorID',
 'McMurray_Top_HorID',
 'McMurray_Base_DEPTH',
 'McMurray_Top_DEPTH',
 'McMurray_Base_Qual',
 'McMurray_Top_Qual',
 'lat',
 'lng',
 'NN1_McMurray_Top_DEPTH',
 'NN1_McMurray_Base_DEPTH',
 'NN1_thickness',
 'MM_Top_Depth_predBy_NN1thick',
 'HorID',
 'Pick',
 'Quality',
 'HorID_paleoz',
 'Pick_paleoz',
 'Quality_paleoz',
 'diff_TMcM_Pick_v_DEPT',
 'diff_TPal_Pick_v_DEPT',
 'cat_isTopMcMrNearby_known',
 'cat_isTopPalNearby_known',
 'DistFrom_NN1_TopDepth_Abs',
 'NewWell',
 'LastBitWell',
 'TopWellDept',
 'BotWellDept',
 'FromTopWell',
 'FromBotWell',
 'WellThickness',
 'closerToBotOrTop',
 'closTopBotDist',
 'rowsToEdge',
 'GR_min_5winSize_dirAroundMin',
 'GR_min_5winSize_dirA

In [26]:
## NOTE WE ARE LEAVING THE UWI in for now but will take it out after dataframe is split into train/test portions!!!!
train_feat_bigList = [
 'UWI',
 'trainOrTest',
 'CALI',
 'COND',
 'DELT',
 'DENS',
 'DPHI',
 'DPHI:1',
 'DPHI:2',
 'DT',
 'GR',
 'GR:1',
 'GR:2',
 'IL',
 'ILD',
 'ILD:1',
 'ILD:2',
 'ILM',
 'LITH',
 'LLD',
 'LLS',
 'NPHI',
 'PHID',
 'PHIN',
 'RESD',
 'RHOB',
 'RT',
 'SFL',
 'SFLU',
 'SN',
 'SNP',
 'SP',
 'McMurray_Base_Qual',
 'McMurray_Top_Qual',
 'lat',
 'lng',  
 'NN1_thickness',
 'MM_Top_Depth_predBy_NN1thick',
 'Quality',
 'Quality_paleoz',
 'DistFrom_NN1_TopDepth_Abs',
 'BotWellDept',
 'FromTopWell',
 'FromBotWell',
 'WellThickness',
 'rowsToEdge',
 'GR_min_5winSize_dirAroundMin',
 'GR_min_5winSize_dirAboveMin',
 'GR_min_5winSize_dirAroundMax',
 'GR_min_5winSize_dirAboveMax',
 'GR_min_5winSize_dirAroundMean',
 'GR_min_5winSize_dirAboveMean',
 'GR_min_5winSize_dirAbovenLarge',
 'GR_min_5winSize_dirAroundnLarge',
 'GR_min_7winSize_dirAroundMin',
 'GR_min_7winSize_dirAboveMin',
 'GR_min_7winSize_dirAroundMax',
 'GR_min_7winSize_dirAboveMax',
 'GR_min_7winSize_dirAroundMean',
 'GR_min_7winSize_dirAboveMean',
 'GR_min_7winSize_dirAbovenLarge',
 'GR_min_7winSize_dirAroundnLarge',
 'GR_min_11winSize_dirAroundMin',
 'GR_min_11winSize_dirAboveMin',
 'GR_min_11winSize_dirAroundMax',
 'GR_min_11winSize_dirAboveMax',
 'GR_min_11winSize_dirAroundMean',
 'GR_min_11winSize_dirAboveMean',
 'GR_min_11winSize_dirAbovenLarge',
 'GR_min_11winSize_dirAroundnLarge',
 'GR_min_21winSize_dirAroundMin',
 'GR_min_21winSize_dirAboveMin',
 'GR_min_21winSize_dirAroundMax',
 'GR_min_21winSize_dirAboveMax',
 'GR_min_21winSize_dirAroundMean',
 'GR_min_21winSize_dirAboveMean',
 'GR_min_21winSize_dirAbovenLarge',
 'GR_min_21winSize_dirAroundnLarge',
 'ILD_min_5winSize_dirAroundMin',
 'ILD_min_5winSize_dirAboveMin',
 'ILD_min_5winSize_dirAroundMax',
 'ILD_min_5winSize_dirAboveMax',
 'ILD_min_5winSize_dirAroundMean',
 'ILD_min_5winSize_dirAboveMean',
 'ILD_min_5winSize_dirAbovenLarge',
 'ILD_min_5winSize_dirAroundnLarge',
 'ILD_min_7winSize_dirAroundMin',
 'ILD_min_7winSize_dirAboveMin',
 'ILD_min_7winSize_dirAroundMax',
 'ILD_min_7winSize_dirAboveMax',
 'ILD_min_7winSize_dirAroundMean',
 'ILD_min_7winSize_dirAboveMean',
 'ILD_min_7winSize_dirAbovenLarge',
 'ILD_min_7winSize_dirAroundnLarge',
 'ILD_min_11winSize_dirAroundMin',
 'ILD_min_11winSize_dirAboveMin',
 'ILD_min_11winSize_dirAroundMax',
 'ILD_min_11winSize_dirAboveMax',
 'ILD_min_11winSize_dirAroundMean',
 'ILD_min_11winSize_dirAboveMean',
 'ILD_min_11winSize_dirAbovenLarge',
 'ILD_min_11winSize_dirAroundnLarge',
 'ILD_min_21winSize_dirAroundMin',
 'ILD_min_21winSize_dirAboveMin',
 'ILD_min_21winSize_dirAroundMax',
 'ILD_min_21winSize_dirAboveMax',
 'ILD_min_21winSize_dirAroundMean',
 'ILD_min_21winSize_dirAboveMean',
 'ILD_min_21winSize_dirAbovenLarge',
 'ILD_min_21winSize_dirAroundnLarge']

In [27]:
len(train_feat_bigList)

110

In [28]:
df_train_feat = df_all_Col_preSplit_wTrainTest_ClassBalanced[train_feat_bigList]

In [29]:
df_train_feat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 208758 entries, 0 to 1482750
Columns: 110 entries, UWI to ILD_min_21winSize_dirAroundnLarge
dtypes: float64(103), int64(5), object(2)
memory usage: 176.8+ MB


In [30]:
df_train_feat.head()

,UWI,trainOrTest,CALI,COND,DELT,DENS,DPHI,DPHI:1,DPHI:2,DT,GR,GR:1,GR:2,IL,ILD,ILD:1,ILD:2,ILM,LITH,LLD,LLS,NPHI,PHID,PHIN,RESD,RHOB,RT,SFL,SFLU,SN,SNP,SP,McMurray_Base_Qual,McMurray_Top_Qual,lat,lng,NN1_thickness,MM_Top_Depth_predBy_NN1thick,Quality,Quality_paleoz,DistFrom_NN1_TopDepth_Abs,BotWellDept,FromTopWell,FromBotWell,WellThickness,rowsToEdge,GR_min_5winSize_dirAroundMin,GR_min_5winSize_dirAboveMin,GR_min_5winSize_dirAroundMax,GR_min_5winSize_dirAboveMax,GR_min_5winSize_dirAroundMean,GR_min_5winSize_dirAboveMean,GR_min_5winSize_dirAbovenLarge,GR_min_5winSize_dirAroundnLarge,GR_min_7winSize_dirAroundMin,GR_min_7winSize_dirAboveMin,GR_min_7winSize_dirAroundMax,GR_min_7winSize_dirAboveMax,GR_min_7winSize_dirAroundMean,GR_min_7winSize_dirAboveMean,GR_min_7winSize_dirAbovenLarge,GR_min_7winSize_dirAroundnLarge,GR_min_11winSize_dirAroundMin,GR_min_11winSize_dirAboveMin,GR_min_11winSize_dirAroundMax,GR_min_11winSize_dirAboveMax,GR_min_11winSize_dirAroundMean,GR_min_11winSize_dirAboveMean,GR_min_11winSize_dirAbovenLarge,GR_min_11winSize_dirAroundnLarge,GR_min_21winSize_dirAroundMin,GR_min_21winSize_dirAboveMin,GR_min_21winSize_dirAroundMax,GR_min_21winSize_dirAboveMax,GR_min_21winSize_dirAroundMean,GR_min_21winSize_dirAboveMean,GR_min_21winSize_dirAbovenLarge,GR_min_21winSize_dirAroundnLarge,ILD_min_5winSize_dirAroundMin,ILD_min_5winSize_dirAboveMin,ILD_min_5winSize_dirAroundMax,ILD_min_5winSize_dirAboveMax,ILD_min_5winSize_dirAroundMean,ILD_min_5winSize_dirAboveMean,ILD_min_5winSize_dirAbovenLarge,ILD_min_5winSize_dirAroundnLarge,ILD_min_7winSize_dirAroundMin,ILD_min_7winSize_dirAboveMin,ILD_min_7winSize_dirAroundMax,ILD_min_7winSize_dirAboveMax,ILD_min_7winSize_dirAroundMean,ILD_min_7winSize_dirAboveMean,ILD_min_7winSize_dirAbovenLarge,ILD_min_7winSize_dirAroundnLarge,ILD_min_11winSize_dirAroundMin,ILD_min_11winSize_dirAboveMin,ILD_min_11winSize_dirAroundMax,ILD_min_11winSize_dirAboveMax,ILD_min_11winSize_dirAroundMean,ILD_min_11winSize_dirAboveMean,ILD_min_11winSize_dirAbovenLarge,ILD_min_11winSize_dirAroundnLarge,ILD_min_21winSize_dirAroundMin,ILD_min_21winSize_dirAboveMin,ILD_min_21winSize_dirAroundMax,ILD_min_21winSize_dirAboveMax,ILD_min_21winSize_dirAroundMean,ILD_min_21winSize_dirAboveMean,ILD_min_21winSize_dirAbovenLarge,ILD_min_21winSize_dirAroundnLarge
0,00/10-32-080-20W4/0,train,167.003,NaN,NaN,NaN,0.227,NaN,NaN,NaN,102.473,NaN,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN,NaN,0.460,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3,55.978836,-113.095365,25.0,359.66,3,1,210.058,396.102,0.0,246.5,246.5,0,102.473,102.473,102.473,102.473,102.4730,102.4730,102.4730,102.4730,102.473,102.473,102.473,102.473,102.473000,102.473000,102.4730,102.4730,102.473,102.473,102.473,102.473,102.473000,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,0.000,0.000,0.000,0.000,0.0000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000,0.000000,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000,0.000000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
10,00/10-32-080-20W4/0,train,166.675,NaN,NaN,NaN,0.269,NaN,NaN,NaN,26.625,NaN,NaN,NaN,30.179,NaN,NaN,NaN,NaN,NaN,NaN,0.355,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3,55.978836,-113.095365,25.0,359.66,3,1,207.558,396.102,2.5,244.0,246.5,10,25.825,26.625,50.213,26.625,32.7680,26.6250,26.6250,26.6250,26.625,26.625,26.625,26.625,26.625000,26.625000,26.6250,26.6250,26.625,26.625,26.625,26.625,26.625000,26.625,26.625,26.625,26.625,26.625,26.625,26.625,26.625,26.625,26.625,26.625,20.262,30.179,30.370,30.179,27.1088,30.1790,30.1790,30.1790,30.179,30.179,30.179,30.179,30.179000,30.179000,30.1790,30.1790,30.179,30.179,30.179,30.179,30.179000,30.179,30.179,30.179,30.179,30.179,30.179,30.179,30.179,30.179,30.179,30.179
20,00/10-32-080-20W4/0,train,211.701,NaN,NaN,NaN,0.339,NaN,NaN,NaN,31.562,NaN,NaN,NaN,21.793,NaN,NaN,NaN,NaN,NaN,NaN,0.428,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3,55.978836,-113.095365,25.0,359.66,3,1,205.058,396.102,5.0,241.5,246.5,20,23.605,31.562,49.258,

In [31]:
training_feats_w_lowCount = ['COND','DELT','DENS','DPHI:1','DPHI:2','DT','GR:1','GR:2','IL','ILD:1','ILD:2','ILM','LITH','LLD','LLS','PHID','PHIN','RESD','RT','SFL','SFLU','SN','SNP','Sp']

In [32]:
train_feats_minusLowCount = [x for x in train_feat_bigList if x not in training_feats_w_lowCount]

In [33]:
len(train_feats_minusLowCount)

87

In [34]:
df_train_featWithHighCount = df_train_feat[train_feats_minusLowCount]

In [35]:
df_train_featWithHighCount.head()

,UWI,trainOrTest,CALI,DPHI,GR,ILD,NPHI,RHOB,SP,McMurray_Base_Qual,McMurray_Top_Qual,lat,lng,NN1_thickness,MM_Top_Depth_predBy_NN1thick,Quality,Quality_paleoz,DistFrom_NN1_TopDepth_Abs,BotWellDept,FromTopWell,FromBotWell,WellThickness,rowsToEdge,GR_min_5winSize_dirAroundMin,GR_min_5winSize_dirAboveMin,GR_min_5winSize_dirAroundMax,GR_min_5winSize_dirAboveMax,GR_min_5winSize_dirAroundMean,GR_min_5winSize_dirAboveMean,GR_min_5winSize_dirAbovenLarge,GR_min_5winSize_dirAroundnLarge,GR_min_7winSize_dirAroundMin,GR_min_7winSize_dirAboveMin,GR_min_7winSize_dirAroundMax,GR_min_7winSize_dirAboveMax,GR_min_7winSize_dirAroundMean,GR_min_7winSize_dirAboveMean,GR_min_7winSize_dirAbovenLarge,GR_min_7winSize_dirAroundnLarge,GR_min_11winSize_dirAroundMin,GR_min_11winSize_dirAboveMin,GR_min_11winSize_dirAroundMax,GR_min_11winSize_dirAboveMax,GR_min_11winSize_dirAroundMean,GR_min_11winSize_dirAboveMean,GR_min_11winSize_dirAbovenLarge,GR_min_11winSize_dirAroundnLarge,GR_min_21winSize_dirAroundMin,GR_min_21winSize_dirAboveMin,GR_min_21winSize_dirAroundMax,GR_min_21winSize_dirAboveMax,GR_min_21winSize_dirAroundMean,GR_min_21winSize_dirAboveMean,GR_min_21winSize_dirAbovenLarge,GR_min_21winSize_dirAroundnLarge,ILD_min_5winSize_dirAroundMin,ILD_min_5winSize_dirAboveMin,ILD_min_5winSize_dirAroundMax,ILD_min_5winSize_dirAboveMax,ILD_min_5winSize_dirAroundMean,ILD_min_5winSize_dirAboveMean,ILD_min_5winSize_dirAbovenLarge,ILD_min_5winSize_dirAroundnLarge,ILD_min_7winSize_dirAroundMin,ILD_min_7winSize_dirAboveMin,ILD_min_7winSize_dirAroundMax,ILD_min_7winSize_dirAboveMax,ILD_min_7winSize_dirAroundMean,ILD_min_7winSize_dirAboveMean,ILD_min_7winSize_dirAbovenLarge,ILD_min_7winSize_dirAroundnLarge,ILD_min_11winSize_dirAroundMin,ILD_min_11winSize_dirAboveMin,ILD_min_11winSize_dirAroundMax,ILD_min_11winSize_dirAboveMax,ILD_min_11winSize_dirAroundMean,ILD_min_11winSize_dirAboveMean,ILD_min_11winSize_dirAbovenLarge,ILD_min_11winSize_dirAroundnLarge,ILD_min_21winSize_dirAroundMin,ILD_min_21winSize_dirAboveMin,ILD_min_21winSize_dirAroundMax,ILD_min_21winSize_dirAboveMax,ILD_min_21winSize_dirAroundMean,ILD_min_21winSize_dirAboveMean,ILD_min_21winSize_dirAbovenLarge,ILD_min_21winSize_dirAroundnLarge
0,00/10-32-080-20W4/0,train,167.003,0.227,102.473,0.000,0.460,NaN,NaN,1,3,55.978836,-113.095365,25.0,359.66,3,1,210.058,396.102,0.0,246.5,246.5,0,102.473,102.473,102.473,102.473,102.4730,102.4730,102.4730,102.4730,102.473,102.473,102.473,102.473,102.473000,102.473000,102.4730,102.4730,102.473,102.473,102.473,102.473,102.473000,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,0.000,0.000,0.000,0.000,0.0000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000,0.000000,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000,0.000000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
10,00/10-32-080-20W4/0,train,166.675,0.269,26.625,30.179,0.355,NaN,NaN,1,3,55.978836,-113.095365,25.0,359.66,3,1,207.558,396.102,2.5,244.0,246.5,10,25.825,26.625,50.213,26.625,32.7680,26.6250,26.6250,26.6250,26.625,26.625,26.625,26.625,26.625000,26.625000,26.6250,26.6250,26.625,26.625,26.625,26.625,26.625000,26.625,26.625,26.625,26.625,26.625,26.625,26.625,26.625,26.625,26.625,26.625,20.262,30.179,30.370,30.179,27.1088,30.1790,30.1790,30.1790,30.179,30.179,30.179,30.179,30.179000,30.179000,30.1790,30.1790,30.179,30.179,30.179,30.179,30.179000,30.179,30.179,30.179,30.179,30.179,30.179,30.179,30.179,30.179,30.179,30.179
20,00/10-32-080-20W4/0,train,211.701,0.339,31.562,21.793,0.428,NaN,NaN,1,3,55.978836,-113.095365,25.0,359.66,3,1,205.058,396.102,5.0,241.5,246.5,20,23.605,31.562,49.258,31.562,34.0164,31.5620,31.5620,31.5620,22.700,31.562,60.528,31.562,36.187143,31.562000,31.5620,31.5620,31.562,31.562,31.562,31.562,31.562000,31.562,31.562,31.562,31.562,31.562,31.562,31.562,31.562,31.562,31.562,31.562,16.583,21.793,25.975,21.793,21.5708,21.7930,21.7930,21.7930,14.586,21.793,26.774,21.793,21.316286,21.793000,21.7930,21.7930,21.793,21.793,21.793,21.793,21.793000,21.7

-----------------

## Identify which columns to use as labels

#### The column 'cat_isTopMcMrNearby_known' is what we'll use as labels.
##### 100 = exactly the Top McMurray Pick
##### 95 if the distance between that depth and the Top McMurray Pick is -0.5 < x and x <0.5
##### 60 if the distance between that depth and the Top McMurray Pick is -5 < x and x < 5
##### 0 = not near the Top McMurray Pick
`df_all_wells_wKNN_DEPTHtoDEPT['cat_isTopMcMrNearby_known']=df_all_wells_wKNN_DEPTHtoDEPT['diff_TMcM_Pick_v_DEPT'].apply(lambda x: 100 if x==0 else ( 95 if (-0.5 < x and x <0.5) else 60 if (-5 < x and x <5) else 0))`

In [36]:
df_all_Col_preSplit_wTrainTest_ClassBalanced['cat_isTopMcMrNearby_known'].unique()

array([  0,  60,  95, 100])

In [37]:
labels = df_all_Col_preSplit_wTrainTest_ClassBalanced[['cat_isTopMcMrNearby_known','UWI','trainOrTest']]

In [38]:
labels.head()

,cat_isTopMcMrNearby_known,UWI,trainOrTest
0,0,00/10-32-080-20W4/0,train
10,0,00/10-32-080-20W4/0,train
20,0,00/10-32-080-20W4/0,train
30,0,00/10-32-080-20W4/0,train
40,0,00/10-32-080-20W4/0,train


In [39]:
labels.tail()

,cat_isTopMcMrNearby_known,UWI,trainOrTest
1482710,0,00/10-35-081-15W4/0,train
1482720,0,00/10-35-081-15W4/0,train
1482730,0,00/10-35-081-15W4/0,train
1482740,0,00/10-35-081-15W4/0,train
1482750,0,00/10-35-081-15W4/0,train


In [76]:
len(labels)

208758

-----------------

## Now separate into 4 dataframes for train_labels, train_feat, test_labels, test_feat
## Then take off UWI and TrainTest col?

In [40]:
# labels_train = labels[labels['trainOrTest'] == 'train' ].drop(columns=['UWI', 'trainOrTest'])
# labels_test = labels[labels['trainOrTest'] == 'test' ].drop(columns=['UWI', 'trainOrTest'])

In [41]:
labels_train = labels[labels['trainOrTest'] == 'train' ]
labels_train = labels_train['cat_isTopMcMrNearby_known']
labels_test = labels[labels['trainOrTest'] == 'test' ]
labels_test = labels_test['cat_isTopMcMrNearby_known']

In [42]:
df_train_featWithHighCount_train = df_train_featWithHighCount[df_train_featWithHighCount['trainOrTest'] == 'train' ].drop(columns=['UWI', 'trainOrTest'])
df_train_featWithHighCount_test = df_train_featWithHighCount[df_train_featWithHighCount['trainOrTest'] == 'test' ].drop(columns=['UWI', 'trainOrTest'])

In [43]:
train_X = df_train_featWithHighCount_train
train_y = labels_train
test_X = df_train_featWithHighCount_test
test_y = labels_test

In [77]:
print(len(train_X))
train_X.head()

166945


,CALI,DPHI,GR,ILD,NPHI,RHOB,SP,McMurray_Base_Qual,McMurray_Top_Qual,lat,lng,NN1_thickness,MM_Top_Depth_predBy_NN1thick,Quality,Quality_paleoz,DistFrom_NN1_TopDepth_Abs,BotWellDept,FromTopWell,FromBotWell,WellThickness,rowsToEdge,GR_min_5winSize_dirAroundMin,GR_min_5winSize_dirAboveMin,GR_min_5winSize_dirAroundMax,GR_min_5winSize_dirAboveMax,GR_min_5winSize_dirAroundMean,GR_min_5winSize_dirAboveMean,GR_min_5winSize_dirAbovenLarge,GR_min_5winSize_dirAroundnLarge,GR_min_7winSize_dirAroundMin,GR_min_7winSize_dirAboveMin,GR_min_7winSize_dirAroundMax,GR_min_7winSize_dirAboveMax,GR_min_7winSize_dirAroundMean,GR_min_7winSize_dirAboveMean,GR_min_7winSize_dirAbovenLarge,GR_min_7winSize_dirAroundnLarge,GR_min_11winSize_dirAroundMin,GR_min_11winSize_dirAboveMin,GR_min_11winSize_dirAroundMax,GR_min_11winSize_dirAboveMax,GR_min_11winSize_dirAroundMean,GR_min_11winSize_dirAboveMean,GR_min_11winSize_dirAbovenLarge,GR_min_11winSize_dirAroundnLarge,GR_min_21winSize_dirAroundMin,GR_min_21winSize_dirAboveMin,GR_min_21winSize_dirAroundMax,GR_min_21winSize_dirAboveMax,GR_min_21winSize_dirAroundMean,GR_min_21winSize_dirAboveMean,GR_min_21winSize_dirAbovenLarge,GR_min_21winSize_dirAroundnLarge,ILD_min_5winSize_dirAroundMin,ILD_min_5winSize_dirAboveMin,ILD_min_5winSize_dirAroundMax,ILD_min_5winSize_dirAboveMax,ILD_min_5winSize_dirAroundMean,ILD_min_5winSize_dirAboveMean,ILD_min_5winSize_dirAbovenLarge,ILD_min_5winSize_dirAroundnLarge,ILD_min_7winSize_dirAroundMin,ILD_min_7winSize_dirAboveMin,ILD_min_7winSize_dirAroundMax,ILD_min_7winSize_dirAboveMax,ILD_min_7winSize_dirAroundMean,ILD_min_7winSize_dirAboveMean,ILD_min_7winSize_dirAbovenLarge,ILD_min_7winSize_dirAroundnLarge,ILD_min_11winSize_dirAroundMin,ILD_min_11winSize_dirAboveMin,ILD_min_11winSize_dirAroundMax,ILD_min_11winSize_dirAboveMax,ILD_min_11winSize_dirAroundMean,ILD_min_11winSize_dirAboveMean,ILD_min_11winSize_dirAbovenLarge,ILD_min_11winSize_dirAroundnLarge,ILD_min_21winSize_dirAroundMin,ILD_min_21winSize_dirAboveMin,ILD_min_21winSize_dirAroundMax,ILD_min_21winSize_dirAboveMax,ILD_min_21winSize_dirAroundMean,ILD_min_21winSize_dirAboveMean,ILD_min_21winSize_dirAbovenLarge,ILD_min_21winSize_dirAroundnLarge
0,167.003,0.227,102.473,0.000,0.460,NaN,NaN,1,3,55.978836,-113.095365,25.0,359.66,3,1,210.058,396.102,0.0,246.5,246.5,0,102.473,102.473,102.473,102.473,102.4730,102.4730,102.4730,102.4730,102.473,102.473,102.473,102.473,102.473000,102.473000,102.4730,102.4730,102.473,102.473,102.473,102.473,102.473000,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,0.000,0.000,0.000,0.000,0.0000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000,0.000000,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000,0.000000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
10,166.675,0.269,26.625,30.179,0.355,NaN,NaN,1,3,55.978836,-113.095365,25.0,359.66,3,1,207.558,396.102,2.5,244.0,246.5,10,25.825,26.625,50.213,26.625,32.7680,26.6250,26.6250,26.6250,26.625,26.625,26.625,26.625,26.625000,26.625000,26.6250,26.6250,26.625,26.625,26.625,26.625,26.625000,26.625,26.625,26.625,26.625,26.625,26.625,26.625,26.625,26.625,26.625,26.625,20.262,30.179,30.370,30.179,27.1088,30.1790,30.1790,30.1790,30.179,30.179,30.179,30.179,30.179000,30.179000,30.1790,30.1790,30.179,30.179,30.179,30.179,30.179000,30.179,30.179,30.179,30.179,30.179,30.179,30.179,30.179,30.179,30.179,30.179
20,211.701,0.339,31.562,21.793,0.428,NaN,NaN,1,3,55.978836,-113.095365,25.0,359.66,3,1,205.058,396.102,5.0,241.5,246.5,20,23.605,31.562,49.258,31.562,34.0164,31.5620,31.5620,31.5620,22.700,31.562,60.528,31.562,36.187143,31.562000,31.5620,31.5620,31.562,31.562,31.562,31.562,31.562000,31.562,31.562,31.562,31.562,31.562,31.562,31.562,31.562,31.562,31.562,31.562,16.583,21.793,25.975,21.793,21.5708,21.7930,21.7930,21.7930,14.586,21.793,26.774,21.793,21.316286,21.793000,21.7930,21.7930,21.793,21.793,21.793,21.793,21.793000,21.793,21.793,21.793,21.793,21.793,21.793,21.793,21.793,21.793,21.793,21.793
30,188.132,0.291,51.2

In [78]:
print(len(train_y))
train_y.head()

166945


0     0
10    0
20    0
30    0
40    0
Name: cat_isTopMcMrNearby_known, dtype: int64

In [79]:
print(len(test_X))
test_X.head()

41813


,CALI,DPHI,GR,ILD,NPHI,RHOB,SP,McMurray_Base_Qual,McMurray_Top_Qual,lat,lng,NN1_thickness,MM_Top_Depth_predBy_NN1thick,Quality,Quality_paleoz,DistFrom_NN1_TopDepth_Abs,BotWellDept,FromTopWell,FromBotWell,WellThickness,rowsToEdge,GR_min_5winSize_dirAroundMin,GR_min_5winSize_dirAboveMin,GR_min_5winSize_dirAroundMax,GR_min_5winSize_dirAboveMax,GR_min_5winSize_dirAroundMean,GR_min_5winSize_dirAboveMean,GR_min_5winSize_dirAbovenLarge,GR_min_5winSize_dirAroundnLarge,GR_min_7winSize_dirAroundMin,GR_min_7winSize_dirAboveMin,GR_min_7winSize_dirAroundMax,GR_min_7winSize_dirAboveMax,GR_min_7winSize_dirAroundMean,GR_min_7winSize_dirAboveMean,GR_min_7winSize_dirAbovenLarge,GR_min_7winSize_dirAroundnLarge,GR_min_11winSize_dirAroundMin,GR_min_11winSize_dirAboveMin,GR_min_11winSize_dirAroundMax,GR_min_11winSize_dirAboveMax,GR_min_11winSize_dirAroundMean,GR_min_11winSize_dirAboveMean,GR_min_11winSize_dirAbovenLarge,GR_min_11winSize_dirAroundnLarge,GR_min_21winSize_dirAroundMin,GR_min_21winSize_dirAboveMin,GR_min_21winSize_dirAroundMax,GR_min_21winSize_dirAboveMax,GR_min_21winSize_dirAroundMean,GR_min_21winSize_dirAboveMean,GR_min_21winSize_dirAbovenLarge,GR_min_21winSize_dirAroundnLarge,ILD_min_5winSize_dirAroundMin,ILD_min_5winSize_dirAboveMin,ILD_min_5winSize_dirAroundMax,ILD_min_5winSize_dirAboveMax,ILD_min_5winSize_dirAroundMean,ILD_min_5winSize_dirAboveMean,ILD_min_5winSize_dirAbovenLarge,ILD_min_5winSize_dirAroundnLarge,ILD_min_7winSize_dirAroundMin,ILD_min_7winSize_dirAboveMin,ILD_min_7winSize_dirAroundMax,ILD_min_7winSize_dirAboveMax,ILD_min_7winSize_dirAroundMean,ILD_min_7winSize_dirAboveMean,ILD_min_7winSize_dirAbovenLarge,ILD_min_7winSize_dirAroundnLarge,ILD_min_11winSize_dirAroundMin,ILD_min_11winSize_dirAboveMin,ILD_min_11winSize_dirAroundMax,ILD_min_11winSize_dirAboveMax,ILD_min_11winSize_dirAroundMean,ILD_min_11winSize_dirAboveMean,ILD_min_11winSize_dirAbovenLarge,ILD_min_11winSize_dirAroundnLarge,ILD_min_21winSize_dirAroundMin,ILD_min_21winSize_dirAboveMin,ILD_min_21winSize_dirAroundMax,ILD_min_21winSize_dirAboveMax,ILD_min_21winSize_dirAroundMean,ILD_min_21winSize_dirAboveMean,ILD_min_21winSize_dirAbovenLarge,ILD_min_21winSize_dirAroundnLarge
990,NaN,0.258,123.954,3.665,0.579,NaN,NaN,1,3,56.015445,-111.333199,46.0,431.32,3,1,186.73,487.34,0.75,242.75,243.5,3,123.954,123.954,123.954,123.954,123.9540,123.9540,123.9540,123.9540,123.954,123.954,123.954,123.954,123.954000,123.954000,123.9540,123.9540,123.954,123.954,123.954,123.954,123.954000,123.954,123.954,123.954,123.954,123.954,123.954,123.954,123.95400,123.954,123.954,123.954,3.665,3.665,3.665,3.665,3.6650,3.6650,3.6650,3.6650,3.665,3.665,3.665,3.665,3.665000,3.665000,3.6650,3.6650,3.665,3.665,3.665,3.665,3.665000,3.665,3.665,3.665,3.665,3.665,3.665,3.665,3.66500,3.665,3.665,3.665
1000,NaN,0.280,123.234,3.619,0.596,NaN,NaN,1,3,56.015445,-111.333199,46.0,431.32,3,1,184.23,487.34,3.25,240.25,243.5,13,115.836,123.234,126.644,123.234,121.4082,123.2340,123.2340,123.2340,115.836,123.234,126.644,123.234,121.248286,123.234000,123.2340,123.2340,123.234,123.234,123.234,123.234,123.234000,123.234,123.234,123.234,123.234,123.234,123.234,123.234,123.23400,123.234,123.234,123.234,3.584,3.619,3.666,3.619,3.6280,3.6190,3.6190,3.6190,3.584,3.619,3.721,3.619,3.648571,3.619000,3.6190,3.6190,3.619,3.619,3.619,3.619,3.619000,3.619,3.619,3.619,3.619,3.619,3.619,3.619,3.61900,3.619,3.619,3.619
1010,NaN,0.273,112.040,3.951,0.561,NaN,NaN,1,3,56.015445,-111.333199,46.0,431.32,3,1,181.73,487.34,5.75,237.75,243.5,23,110.998,110.998,112.426,115.666,111.8636,113.0700,113.0700,111.8636,110.630,112.040,114.443,112.040,112.055857,112.040000,112.0400,112.0400,110.492,112.040,115.969,112.040,112.553727,112.040,112.040,112.040,112.040,112.040,112.040,112.040,112.04000,112.040,112.040,112.040,3.890,3.868,3.951,3.951,3.9372,3.9082,3.9082,3.9372,3.868,3.951,3.957,3.951,3.930143,3.951000,3.9510,3.9510,3.807,3.951,3.957,3.951,3.909091,3.951,3.951,3.951,3.951,3.951,3.951,3.951,3.95100,3.951,3.951,3.951
1020,NaN,0.316,

In [80]:
print(len(test_y))
test_y.head()

41813


990     0
1000    0
1010    0
1020    0
1030    0
Name: cat_isTopMcMrNearby_known, dtype: int64

-------------------

# Machine-learning

In [44]:

seed = 123

In [49]:
# .values.ravel()
model = XGBClassifier(
    gamma=0, 
    reg_alpha=0.2, 
    max_depth=3, 
    subsample=0.8, 
    colsample_bytree= 0.8, 
    n_estimators= 300, 
    learning_rate= 0.03, 
    min_child_weight= 3,n_jobs=8)
model.fit(train_X,train_y)
result = model.predict(test_X)
result

array([0, 0, 0, ..., 0, 0, 0])

In [59]:
type(result)

numpy.ndarray

In [67]:
len(result)

41813

In [70]:
df_r = pd.DataFrame(result)
# df_a = pd.DataFrame(a_)
test_y_indexValues = test_y.index.values
df_result_2 = pd.DataFrame(values, index=test_y_indexValues, columns=['TopMcMr_Pick_pred'])
df_results_2 = pd.concat([test_y, df_result_2], axis=1)

In [88]:
df_results_2

,cat_isTopMcMrNearby_known,TopMcMr_Pick_pred
990,0,0
1000,0,0
1010,0,0
1020,0,0
1030,0,0
1040,0,0
1050,0,0
1060,0,0
1070,0,0
1080,0,0


In [57]:
# test_df_pred = test_y.copy()
# test_df_pred['Pick_pred'] = result
# test_df_pred.head()

990     0
1000    0
1010    0
1020    0
1030    0
Name: cat_isTopMcMrNearby_known, dtype: object

In [72]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# make predictions for test data
# y_pred = model.predict(X_test)
# predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(df_results_2['cat_isTopMcMrNearby_known'], df_results_2['TopMcMr_Pick_pred'])

In [73]:
accuracy

0.86217205175423917

--------------

In [150]:
df_results_2_onlyClass95and100 = df_results_2[df_results_2['cat_isTopMcMrNearby_known'] > 60 ]

In [151]:
df_results_2_onlyClass95and100.head()

,cat_isTopMcMrNearby_known,TopMcMr_Pick_pred
1707,95,60
1708,95,60
1709,95,60
1710,95,60
13780,95,60


In [ ]:
df_results_2_onlyClass95and100['pred_class_95or100'] = df_results_2_onlyClass95and100[df_results_2_onlyClass95and100['cat_isTopMcMrNearby_known']]

In [153]:
df_results_2_onlyClass95and100['cat_isTopMcMrNearby_known_95or100'] = np.where(df_results_2_onlyClass95and100['cat_isTopMcMrNearby_known']>60, 1, 0)
df_results_2_onlyClass95and100['TopMcMr_Pick_pred_95or100'] = np.where(df_results_2_onlyClass95and100['TopMcMr_Pick_pred']>60, 1, 0)

/Users/justingosses/anaconda/envs/MannvilleDask/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/justingosses/anaconda/envs/MannvilleDask/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [158]:
df_results_2_onlyClass95and100[660:900]

,cat_isTopMcMrNearby_known,TopMcMr_Pick_pred,cat_isTopMcMrNearby_known_95or100,TopMcMr_Pick_pred_95or100
918221,95,60,1,0
918222,95,60,1,0
920238,95,60,1,0
921775,95,60,1,0
921776,100,60,1,0
921777,95,60,1,0
922820,95,60,1,0
922821,100,60,1,0
922822,95,0,1,0
926206,95,60,1,0


In [156]:
accuracy = accuracy_score(df_results_2_onlyClass95and100['cat_isTopMcMrNearby_known_95or100'], df_results_2_onlyClass95and100['TopMcMr_Pick_pred_95or100'])
accuracy

0.0

## How many times do we predict the closest row to the pick is not a "not be pick" or zero

In [83]:
df_featPlus_wUWI_testCopy = df_train_featWithHighCount[df_train_featWithHighCount['trainOrTest'] == 'test' ].copy()

In [84]:
df_featPlus_wUWI_testCopy.head()

,UWI,trainOrTest,CALI,DPHI,GR,ILD,NPHI,RHOB,SP,McMurray_Base_Qual,McMurray_Top_Qual,lat,lng,NN1_thickness,MM_Top_Depth_predBy_NN1thick,Quality,Quality_paleoz,DistFrom_NN1_TopDepth_Abs,BotWellDept,FromTopWell,FromBotWell,WellThickness,rowsToEdge,GR_min_5winSize_dirAroundMin,GR_min_5winSize_dirAboveMin,GR_min_5winSize_dirAroundMax,GR_min_5winSize_dirAboveMax,GR_min_5winSize_dirAroundMean,GR_min_5winSize_dirAboveMean,GR_min_5winSize_dirAbovenLarge,GR_min_5winSize_dirAroundnLarge,GR_min_7winSize_dirAroundMin,GR_min_7winSize_dirAboveMin,GR_min_7winSize_dirAroundMax,GR_min_7winSize_dirAboveMax,GR_min_7winSize_dirAroundMean,GR_min_7winSize_dirAboveMean,GR_min_7winSize_dirAbovenLarge,GR_min_7winSize_dirAroundnLarge,GR_min_11winSize_dirAroundMin,GR_min_11winSize_dirAboveMin,GR_min_11winSize_dirAroundMax,GR_min_11winSize_dirAboveMax,GR_min_11winSize_dirAroundMean,GR_min_11winSize_dirAboveMean,GR_min_11winSize_dirAbovenLarge,GR_min_11winSize_dirAroundnLarge,GR_min_21winSize_dirAroundMin,GR_min_21winSize_dirAboveMin,GR_min_21winSize_dirAroundMax,GR_min_21winSize_dirAboveMax,GR_min_21winSize_dirAroundMean,GR_min_21winSize_dirAboveMean,GR_min_21winSize_dirAbovenLarge,GR_min_21winSize_dirAroundnLarge,ILD_min_5winSize_dirAroundMin,ILD_min_5winSize_dirAboveMin,ILD_min_5winSize_dirAroundMax,ILD_min_5winSize_dirAboveMax,ILD_min_5winSize_dirAroundMean,ILD_min_5winSize_dirAboveMean,ILD_min_5winSize_dirAbovenLarge,ILD_min_5winSize_dirAroundnLarge,ILD_min_7winSize_dirAroundMin,ILD_min_7winSize_dirAboveMin,ILD_min_7winSize_dirAroundMax,ILD_min_7winSize_dirAboveMax,ILD_min_7winSize_dirAroundMean,ILD_min_7winSize_dirAboveMean,ILD_min_7winSize_dirAbovenLarge,ILD_min_7winSize_dirAroundnLarge,ILD_min_11winSize_dirAroundMin,ILD_min_11winSize_dirAboveMin,ILD_min_11winSize_dirAroundMax,ILD_min_11winSize_dirAboveMax,ILD_min_11winSize_dirAroundMean,ILD_min_11winSize_dirAboveMean,ILD_min_11winSize_dirAbovenLarge,ILD_min_11winSize_dirAroundnLarge,ILD_min_21winSize_dirAroundMin,ILD_min_21winSize_dirAboveMin,ILD_min_21winSize_dirAroundMax,ILD_min_21winSize_dirAboveMax,ILD_min_21winSize_dirAroundMean,ILD_min_21winSize_dirAboveMean,ILD_min_21winSize_dirAbovenLarge,ILD_min_21winSize_dirAroundnLarge
990,00/04-15-081-09W4/0,test,NaN,0.258,123.954,3.665,0.579,NaN,NaN,1,3,56.015445,-111.333199,46.0,431.32,3,1,186.73,487.34,0.75,242.75,243.5,3,123.954,123.954,123.954,123.954,123.9540,123.9540,123.9540,123.9540,123.954,123.954,123.954,123.954,123.954000,123.954000,123.9540,123.9540,123.954,123.954,123.954,123.954,123.954000,123.954,123.954,123.954,123.954,123.954,123.954,123.954,123.95400,123.954,123.954,123.954,3.665,3.665,3.665,3.665,3.6650,3.6650,3.6650,3.6650,3.665,3.665,3.665,3.665,3.665000,3.665000,3.6650,3.6650,3.665,3.665,3.665,3.665,3.665000,3.665,3.665,3.665,3.665,3.665,3.665,3.665,3.66500,3.665,3.665,3.665
1000,00/04-15-081-09W4/0,test,NaN,0.280,123.234,3.619,0.596,NaN,NaN,1,3,56.015445,-111.333199,46.0,431.32,3,1,184.23,487.34,3.25,240.25,243.5,13,115.836,123.234,126.644,123.234,121.4082,123.2340,123.2340,123.2340,115.836,123.234,126.644,123.234,121.248286,123.234000,123.2340,123.2340,123.234,123.234,123.234,123.234,123.234000,123.234,123.234,123.234,123.234,123.234,123.234,123.234,123.23400,123.234,123.234,123.234,3.584,3.619,3.666,3.619,3.6280,3.6190,3.6190,3.6190,3.584,3.619,3.721,3.619,3.648571,3.619000,3.6190,3.6190,3.619,3.619,3.619,3.619,3.619000,3.619,3.619,3.619,3.619,3.619,3.619,3.619,3.61900,3.619,3.619,3.619
1010,00/04-15-081-09W4/0,test,NaN,0.273,112.040,3.951,0.561,NaN,NaN,1,3,56.015445,-111.333199,46.0,431.32,3,1,181.73,487.34,5.75,237.75,243.5,23,110.998,110.998,112.426,115.666,111.8636,113.0700,113.0700,111.8636,110.630,112.040,114.443,112.040,112.055857,112.040000,112.0400,112.0400,110.492,112.040,115.969,112.040,112.553727,112.040,112.040,112.040,112.040,112.040,112.040,112.040,112.04000,112.040,112.040,112.040,3.890,3.868,3.951,3.951,3.9372,3.9082,3.9082,3.9372,3.868,3.951,3.957,3.951,3.930143,3.951000,3.9510,3.9510,3.807,3.951,3.957,3.951,3

In [90]:
df_featPlus_wUWI_testCopy_wResults = pd.concat([df_featPlus_wUWI_testCopy, df_results_2], axis=1)
df_featPlus_wUWI_testCopy_wResults.tail()

,UWI,trainOrTest,CALI,DPHI,GR,ILD,NPHI,RHOB,SP,McMurray_Base_Qual,McMurray_Top_Qual,lat,lng,NN1_thickness,MM_Top_Depth_predBy_NN1thick,Quality,Quality_paleoz,DistFrom_NN1_TopDepth_Abs,BotWellDept,FromTopWell,FromBotWell,WellThickness,rowsToEdge,GR_min_5winSize_dirAroundMin,GR_min_5winSize_dirAboveMin,GR_min_5winSize_dirAroundMax,GR_min_5winSize_dirAboveMax,GR_min_5winSize_dirAroundMean,GR_min_5winSize_dirAboveMean,GR_min_5winSize_dirAbovenLarge,GR_min_5winSize_dirAroundnLarge,GR_min_7winSize_dirAroundMin,GR_min_7winSize_dirAboveMin,GR_min_7winSize_dirAroundMax,GR_min_7winSize_dirAboveMax,GR_min_7winSize_dirAroundMean,GR_min_7winSize_dirAboveMean,GR_min_7winSize_dirAbovenLarge,GR_min_7winSize_dirAroundnLarge,GR_min_11winSize_dirAroundMin,GR_min_11winSize_dirAboveMin,GR_min_11winSize_dirAroundMax,GR_min_11winSize_dirAboveMax,GR_min_11winSize_dirAroundMean,GR_min_11winSize_dirAboveMean,GR_min_11winSize_dirAbovenLarge,GR_min_11winSize_dirAroundnLarge,GR_min_21winSize_dirAroundMin,GR_min_21winSize_dirAboveMin,GR_min_21winSize_dirAroundMax,GR_min_21winSize_dirAboveMax,GR_min_21winSize_dirAroundMean,GR_min_21winSize_dirAboveMean,GR_min_21winSize_dirAbovenLarge,GR_min_21winSize_dirAroundnLarge,ILD_min_5winSize_dirAroundMin,ILD_min_5winSize_dirAboveMin,ILD_min_5winSize_dirAroundMax,ILD_min_5winSize_dirAboveMax,ILD_min_5winSize_dirAroundMean,ILD_min_5winSize_dirAboveMean,ILD_min_5winSize_dirAbovenLarge,ILD_min_5winSize_dirAroundnLarge,ILD_min_7winSize_dirAroundMin,ILD_min_7winSize_dirAboveMin,ILD_min_7winSize_dirAroundMax,ILD_min_7winSize_dirAboveMax,ILD_min_7winSize_dirAroundMean,ILD_min_7winSize_dirAboveMean,ILD_min_7winSize_dirAbovenLarge,ILD_min_7winSize_dirAroundnLarge,ILD_min_11winSize_dirAroundMin,ILD_min_11winSize_dirAboveMin,ILD_min_11winSize_dirAroundMax,ILD_min_11winSize_dirAboveMax,ILD_min_11winSize_dirAroundMean,ILD_min_11winSize_dirAboveMean,ILD_min_11winSize_dirAbovenLarge,ILD_min_11winSize_dirAroundnLarge,ILD_min_21winSize_dirAroundMin,ILD_min_21winSize_dirAboveMin,ILD_min_21winSize_dirAroundMax,ILD_min_21winSize_dirAboveMax,ILD_min_21winSize_dirAroundMean,ILD_min_21winSize_dirAboveMean,ILD_min_21winSize_dirAbovenLarge,ILD_min_21winSize_dirAroundnLarge,cat_isTopMcMrNearby_known,TopMcMr_Pick_pred
1473050,00/06-22-078-04W4/0,test,NaN,0.308,130.061,3.685,0.621,2183.47,NaN,1,1,55.771708,-110.539294,86.56,306.44,1,1,81.56,400.0,248.0,12.0,260.0,48,120.035,105.744,136.340,136.340,128.6864,127.4644,127.4644,128.6864,120.035,79.523,136.340,136.340,128.454143,113.854000,127.4644,131.6942,80.133,28.332,136.340,136.340,122.504727,92.121909,127.4644,133.044,28.332,130.061,136.340,130.061,99.973762,130.061,130.061,130.061,3.481,3.554,4.130,4.130,3.7786,3.8648,3.8648,3.7786,3.481,2.837,4.130,4.130,3.782429,3.619429,3.8648,3.890,3.175,1.911,4.276,4.130,3.754727,3.084545,3.8648,4.0362,1.911,3.685,4.783,3.685,3.600143,3.685,3.685,3.685,0,0
1473060,00/06-22-078-04W4/0,test,NaN,0.240,34.303,4.348,0.340,2299.34,NaN,1,1,55.771708,-110.539294,86.56,306.44,1,1,84.06,400.0,250.5,9.5,260.0,38,19.898,34.303,93.702,133.728,49.3010,91.2022,91.2022,49.3010,16.760,34.303,119.964,133.728,54.747000,102.643857,121.9780,69.3142,16.743,34.303,133.728,34.303,62.083818,34.303000,34.3030,34.303,34.303,34.303,34.303,34.303,34.303000,34.303,34.303,34.303,3.036,4.348,4.783,4.783,4.1160,4.5740,4.5740,4.1160,2.466,3.820,4.783,4.783,3.963143,4.423714,4.5740,4.448,2.079,4.348,4.783,4.348,3.707182,4.348000,4.3480,4.3480,4.348,4.348,4.348,4.348,4.348000,4.348,4.348,4.348,0,0
1473070,00/06-22-078-04W4/0,test,NaN,0.593,128.531,4.583,0.571,1695.98,NaN,1,1,55.771708,-110.539294,86.56,306.44,1,1,86.56,400.0,253.0,7.0,260.0,28,41.690,22.341,128.531,128.531,92.2838,57.3134,57.3134,92.2838,29.205,128.531,128.531,128.531,83.088000,128.531000,128.5310,128.5310,17.980,128.531,128.531,128.531,71.829909,128.531000,128.5310,128.531,128.531,128.531,128.531,128.531,128.531000,128.531,128.531,128.531,3.144,2.039,6.192,4.583,4.6494,3.2140,3.2140,4.6494,2.508,4.583,6.192,4.583,4.547000

In [126]:
wells_in_test = df_featPlus_wUWI_testCopy_wResults['UWI'].unique()
len(wells_in_test)

382

In [81]:
import pdvega
import vega

In [136]:
df_look_at_pred_class_vs_distFromRealLess1 = df_featPlus_wUWI_testCopy_wResults[df_featPlus_wUWI_testCopy_wResults['DistFrom_NN1_TopDepth_Abs'] < 1 ]

In [137]:
df_look_at_pred_class_vs_distFromRealLess1['cat_isTopMcMrNearby_known'].nunique()

4

In [138]:
df_count = df_look_at_pred_class_vs_distFromRealLess1.groupby('cat_isTopMcMrNearby_known').nunique()

In [139]:
df_count

,UWI,trainOrTest,CALI,DPHI,GR,ILD,NPHI,RHOB,SP,McMurray_Base_Qual,McMurray_Top_Qual,lat,lng,NN1_thickness,MM_Top_Depth_predBy_NN1thick,Quality,Quality_paleoz,DistFrom_NN1_TopDepth_Abs,BotWellDept,FromTopWell,FromBotWell,WellThickness,rowsToEdge,GR_min_5winSize_dirAroundMin,GR_min_5winSize_dirAboveMin,GR_min_5winSize_dirAroundMax,GR_min_5winSize_dirAboveMax,GR_min_5winSize_dirAroundMean,GR_min_5winSize_dirAboveMean,GR_min_5winSize_dirAbovenLarge,GR_min_5winSize_dirAroundnLarge,GR_min_7winSize_dirAroundMin,GR_min_7winSize_dirAboveMin,GR_min_7winSize_dirAroundMax,GR_min_7winSize_dirAboveMax,GR_min_7winSize_dirAroundMean,GR_min_7winSize_dirAboveMean,GR_min_7winSize_dirAbovenLarge,GR_min_7winSize_dirAroundnLarge,GR_min_11winSize_dirAroundMin,GR_min_11winSize_dirAboveMin,GR_min_11winSize_dirAroundMax,GR_min_11winSize_dirAboveMax,GR_min_11winSize_dirAroundMean,GR_min_11winSize_dirAboveMean,GR_min_11winSize_dirAbovenLarge,GR_min_11winSize_dirAroundnLarge,GR_min_21winSize_dirAroundMin,GR_min_21winSize_dirAboveMin,GR_min_21winSize_dirAroundMax,GR_min_21winSize_dirAboveMax,GR_min_21winSize_dirAroundMean,GR_min_21winSize_dirAboveMean,GR_min_21winSize_dirAbovenLarge,GR_min_21winSize_dirAroundnLarge,ILD_min_5winSize_dirAroundMin,ILD_min_5winSize_dirAboveMin,ILD_min_5winSize_dirAroundMax,ILD_min_5winSize_dirAboveMax,ILD_min_5winSize_dirAroundMean,ILD_min_5winSize_dirAboveMean,ILD_min_5winSize_dirAbovenLarge,ILD_min_5winSize_dirAroundnLarge,ILD_min_7winSize_dirAroundMin,ILD_min_7winSize_dirAboveMin,ILD_min_7winSize_dirAroundMax,ILD_min_7winSize_dirAboveMax,ILD_min_7winSize_dirAroundMean,ILD_min_7winSize_dirAboveMean,ILD_min_7winSize_dirAbovenLarge,ILD_min_7winSize_dirAroundnLarge,ILD_min_11winSize_dirAroundMin,ILD_min_11winSize_dirAboveMin,ILD_min_11winSize_dirAroundMax,ILD_min_11winSize_dirAboveMax,ILD_min_11winSize_dirAroundMean,ILD_min_11winSize_dirAboveMean,ILD_min_11winSize_dirAbovenLarge,ILD_min_11winSize_dirAroundnLarge,ILD_min_21winSize_dirAroundMin,ILD_min_21winSize_dirAboveMin,ILD_min_21winSize_dirAroundMax,ILD_min_21winSize_dirAboveMax,ILD_min_21winSize_dirAroundMean,ILD_min_21winSize_dirAboveMean,ILD_min_21winSize_dirAbovenLarge,ILD_min_21winSize_dirAroundnLarge,cat_isTopMcMrNearby_known,TopMcMr_Pick_pred
cat_isTopMcMrNearby_known,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,158,1,50,101,157,155,112,13,1,4,3,157,157,128,153,3,4,87,134,146,137,104,128,157,157,156,157,157,157,157,157,157,156,156,157,157,157,157,157,156,156,157,157,157,157,157,157,156,156,157,157,157,157,157,157,156,155,154,155,156,156,156,156,156,155,156,155,156,156,156,156,156,156,156,156,156,156,156,156,156,156,156,156,156,156,156,156,1,2
60,149,1,275,281,858,825,285,85,8,4,3,149,148,113,146,3,4,381,134,547,435,106,304,586,592,586,588,865,865,865,865,511,535,515,527,865,865,806,794,441,483,447,509,864,864,727,713,384,466,430,485,865,865,645,645,654,671,639,650,850,847,846,846,600,616,566,593,850,846,782,779,530,567,474,530,847,847,705,673,483,526,393,528,846,847,645,565,1,2
95,43,1,34,66,85,84,66,9,0,4,3,43,42,39,42,3,4,53,38,79,79,35,76,77,77,62,73,85,85,85,85,74,71,61,72,85,85,81,81,72,57,64,74,85,85,81,76,53,64,58,74,85,85,79,76,67,74,83,78,84,84,84,84,63,75,83,68,85,84,80,85,60,77,72,63,85,84,70,83,64,76,62,67,84,84,70,69,1,1
100,17,1,8,17,17,17,16,1,0,2,3,17,17,16,17,3,2,6,15,17,17,16,16,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,1,1


In [140]:
total_rows_less_than_1_from_pick = df_count['UWI'].unique().sum()
total_rows_less_than_1_from_pick

367

In [141]:
df_count['UWI']

cat_isTopMcMrNearby_known
0      158
60     149
95      43
100     17
Name: UWI, dtype: int64

In [142]:
def getPercents(df,total_wells):
    for Each in df:
        print(Each, "total or the % is: ",Each/total_wells)
    
    

In [143]:
getPercents(df_count['UWI'],total_rows_less_than_1_from_pick)

158 total or the % is:  0.430517711172
149 total or the % is:  0.405994550409
43 total or the % is:  0.117166212534
17 total or the % is:  0.0463215258856


In [ ]:
# 'MM_Top_Depth_predBy_NN1thick'
test_df_pred2_TopScratch2 = test_df_pred_onlyTopMCM_class5[['DistFrom_NN1_TopDepth']]
test_df_pred2_TopScratch2.vgplot.hist(bins=100, alpha=0.5)

In [ ]:
# client = Client()
# client

In [ ]:
# client.close()

In [ ]:
# from xgboost.sklearn import XGBClassifier
# from dask_ml.xgboost import XGBRegressor


# from dask_ml.xgboost import XGBClassifier

In [ ]:
# train_X_dd = dd.from_pandas(train_X, npartitions=10)
# train_y_dd = dd.from_pandas(train_y, npartitions=10)

In [ ]:
# est = XGBClassifier(gamma=0, 
#     reg_alpha=0.2, 
#     max_depth=3, 
#     subsample=0.8, 
#     colsample_bytree= 0.8, 
#     n_estimators= 300, 
#     learning_rate= 0.03, 
#     min_child_weight= 3)
# est.fit(train_X,train_y)

## What types of normalization are appropriate? 